In [ ]:
!pip install -Uqq fastai fastbook

Paso 1: Descargar imágenes de peces payaso y otros en general

In [ ]:
from fastcore.all import *
from fastbook import *

def search_images(term,max_images=100):
  print(f"Buscando imágenes de '{term}'")
  return search_images_ddg(term,max_images=max_images)
urls = search_images('foto radiografía mamas sanas', max_images=1)
urls[0]


In [ ]:
from fastdownload import download_url
from fastai.vision.all import *
dest='radiografia_por_determinar.jpg'
download_url(urls[0],dest,show_progress=False)
im=Image.open(dest)
im.to_thumb(256,256)



In [ ]:
download_url(search_images('radiografía con cáncer de mama',max_images=1)[0],'radiografía con cáncer de mama.jpg',show_progress=False)
Image.open('radiografía con cáncer de mama.jpg').to_thumb(256,256)

In [ ]:
searches = 'radiografía real mama sana','radiografía real cancer de mama'
path = Path('sano_o_cáncer')
from time import sleep

for o in searches:
  dest= path/o
  dest.mkdir(exist_ok=True, parents=True)
  download_images(dest, urls=search_images(f'{o} foto'))
  sleep(10)  # Pause between searches to avoid over-loading server
  download_images(dest, urls=search_images(f'{o} en hospital'))
  sleep(10)
  download_images(dest, urls=search_images(f'{o} diagnóstico'))
  sleep(10)
  resize_images(path/o, max_size=400, dest=path/o)


Entrenar nuestro modelo

In [ ]:
#verificar si hay imágenesque no se pueden abrir correctamente/dañadas
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
dls = DataBlock(
    blocks = (ImageBlock,CategoryBlock),
    get_items = get_image_files,
    splitter=RandomSplitter(valid_pct=0.2,seed=42),
    get_y = parent_label,
    item_tfms=[Resize(192, method='squish')]
    ).dataloaders(path, bs=32)
dls.show_batch(max_n=6, figsize=(20, 5))


Ahora estamos listos para entrenar nuestro modelo. El modelo utilizado es 'resnet50'. Fastai viene con un método útil llamado 'fine_tune()' que automáticamente utiliza las mejores prácticas para ajustar finamente un modelo pre-entrenado, así que usaremos eso.

In [ ]:
model = vision_learner(dls, resnet18, metrics=error_rate)
model.fine_tune(3)

Vamos a probar nuestro modelo

In [ ]:
es_neum,_,prob = model.predict(PILImage.create('radiografia_por_determinar.jpg'))
print(f"Esto es un: {es_neum}.")
print(f"Probabilidad de que sea un una persona con cáncer de mama: {prob[0]:.4f}")

In [ ]:
model.export('model.pkl')


!pip install nbconvert
!jupyter nbconvert --to html Clasificador: ¿Pez payaso o tropical?.ipynb
